## Q1

In [4]:

'''
Implementation of Tpot Library
'''
# importing required libraries
import numpy as np 
import pandas as pd 
from sklearn import preprocessing 
from sklearn.metrics import mean_squared_error
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split 
import xgboost


In [5]:
# read the dataset
train =  pd.read_csv('sales_train.csv')
test  =  pd.read_csv('sales_test.csv')
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [6]:
train.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

In [7]:
## preprocessing 
### mean imputations 
train['Item_Weight'].fillna((train['Item_Weight'].mean()), inplace=True)
test['Item_Weight'].fillna((test['Item_Weight'].mean()), inplace=True)


train['Outlet_Establishment_Year'] = 2013 - train['Outlet_Establishment_Year'] 
test['Outlet_Establishment_Year'] = 2013 - test['Outlet_Establishment_Year'] 

train['Item_Visibility'] = np.sqrt(train['Item_Visibility'])
test['Item_Visibility'] = np.sqrt(test['Item_Visibility'])



In [8]:
def str2int(txt):
  nums=[str(s) for s in txt if s.isdigit()]
  return int("".join(nums)) if len(nums)>0 else 0

def str2asciisum(txt):
  return sum([ord(c) for c in txt])

In [9]:
#preprocessing of droping cols and converting the type

#droping the cols
test=test.dropna()
test.drop(['Outlet_Size','Item_Fat_Content','Item_Type'],axis=1,inplace=True)

#converting type
test['Outlet_Identifier']=test['Outlet_Identifier'].apply(str2int)
test['Outlet_Location_Type']=test['Outlet_Location_Type'].apply(str2int)
test['Item_Identifier']=test['Item_Identifier'].apply(str2asciisum)
test['Outlet_Type']=test['Outlet_Type'].apply(str2int)


target = train['Item_Outlet_Sales']

#droping the cols
train = train.dropna()
train.drop(['Item_Outlet_Sales','Outlet_Size', 'Item_Fat_Content',
          'Item_Type'], axis=1, inplace=True)

#converting type
train['Outlet_Identifier'] = train['Outlet_Identifier'].apply(str2int)
train['Outlet_Location_Type'] = train['Outlet_Location_Type'].apply(str2int)
train['Item_Identifier'] = train['Item_Identifier'].apply(str2asciisum)
train['Outlet_Type']=train['Outlet_Type'].apply(str2int)


In [10]:
# finally building model using tpot library
# for a sample train it on a sample as it takes a lot of time to train model
train = train[:100]
target = target[:100]

X_train, X_test, y_train, y_test = train_test_split(train, target,
 train_size=0.75, test_size=0.25)

In [11]:
tpot = TPOTRegressor(generations=2, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_boston_pipeline.py')



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.
                                                                             
Best pipeline: DecisionTreeRegressor(input_matrix, max_depth=1, min_samples_leaf=7, min_samples_split=14)
-2508976.7231813893


## Q2

In [12]:

'''
Implementation of Tpot Library
'''
# importing required libraries
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
import xgboost


In [26]:
# read the dataset
train = pd.read_csv('titanic_train.csv')
test = pd.read_csv('titanic_test.csv')

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
train = train.filter(items=['Survived', 'Pclass',
                         'Sex', 'Age', 'Embarked']).dropna()  # trimming the data we need, droping rows with empty col
train = train.filter(items=['Survived', 'Pclass',
                      'Sex', 'Age', 'Embarked']).dropna()  # trimming the data we need, droping rows with empty col
train

,Survived,Pclass,Sex,Age,Embarked
0,0,3,male,22.0,S
1,1,1,female,38.0,C
2,1,3,female,26.0,S
3,1,1,female,35.0,S
4,0,3,male,35.0,S
...,...,...,...,...,...
885,0,3,female,39.0,Q
886,0,2,male,27.0,S
887,1,1,female,19.0,S
889,1,1,male,26.0,C


In [28]:
test = test.filter(items=['Survived', 'Pclass',
                            'Sex', 'Age', 'Embarked']).dropna()  # trimming the data we need, droping rows with empty col
test = test.filter(items=['Survived', 'Pclass',
                            'Sex', 'Age', 'Embarked']).dropna()  # trimming the data we need, droping rows with empty col
test

,Pclass,Sex,Age,Embarked
0,3,male,34.5,Q
1,3,female,47.0,S
2,2,male,62.0,Q
3,3,male,27.0,S
4,3,female,22.0,S
...,...,...,...,...
409,3,female,3.0,S
411,1,female,37.0,Q
412,3,female,28.0,S
414,1,female,39.0,C


In [29]:
#classifying
train['Age'] = train['Age'].apply(lambda age: int(age//10) if age < 60 else 6)
train['Sex'] = train['Sex'].apply(lambda sex: 1 if sex == "male" else 0)
train['Embarked'] = train['Embarked'].apply(lambda x: ['S', 'C', 'Q'].index(x))

train

,Survived,Pclass,Sex,Age,Embarked
0,0,3,1,2,0
1,1,1,0,3,1
2,1,3,0,2,0
3,1,1,0,3,0
4,0,3,1,3,0
...,...,...,...,...,...
885,0,3,0,3,2
886,0,2,1,2,0
887,1,1,0,1,0
889,1,1,1,2,1


In [30]:
#classifying
test['Age'] = test['Age'].apply(lambda age: int(age//10) if age < 60 else 6)
test['Sex'] = test['Sex'].apply(lambda sex: 1 if sex == "male" else 0)
test['Embarked'] = test['Embarked'].apply(lambda x: ['S', 'C', 'Q'].index(x))

test

,Pclass,Sex,Age,Embarked
0,3,1,3,2
1,3,0,4,0
2,2,1,6,2
3,3,1,2,0
4,3,0,2,0
...,...,...,...,...
409,3,0,0,0
411,1,0,3,2
412,3,0,2,0
414,1,0,3,1


In [31]:
target = train['Survived']
train.drop('Survived',axis=1,inplace=True)
print(train.columns)
target


Index(['Pclass', 'Sex', 'Age', 'Embarked'], dtype='object')


0      0
1      1
2      1
3      1
4      0
      ..
885    0
886    0
887    1
889    1
890    0
Name: Survived, Length: 712, dtype: int64

In [32]:
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                    train_size=0.75, test_size=0.25)

In [33]:
tpot = TPOTRegressor(generations=2, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_titanic_pipeline.py')



Generation 1 - Current best internal CV score: -0.14019813254839442

Generation 2 - Current best internal CV score: -0.14001547193773298

Best pipeline: ExtraTreesRegressor(input_matrix, bootstrap=False, max_features=0.8500000000000001, min_samples_leaf=7, min_samples_split=19, n_estimators=100)
-0.1430157498042629
